In [1]:
!pip install fpl
import pandas as pd
import torch
import numpy as np
import random
from random import shuffle
np.random.seed(17)
random.seed(17)
torch.manual_seed(17)
import wget
import os
import aiohttp
import asyncio
from fpl import FPL
from torch.utils.data import TensorDataset, DataLoader
from player import Player
from team import Team
from data_processor import get_fpl, get_current_squad, get_teams, get_players, get_training_datasets
from agent import Agent
import torch.nn as nn
import pytorch_lightning as pl

In [2]:
opponent_feature_names = ["npxG","npxGA"]
player_feature_names = ["total_points", "ict_index", "clean_sheets", "saves", "assists"]
window = 4
teams = get_teams(team_feature_names=opponent_feature_names, visualize=False, window=window)
players = await get_players(player_feature_names, opponent_feature_names, visualize=False, num_players=680, window=window)
train_loader, test_loader, _ = get_training_datasets(players, teams)
train_loader, test_loader

/home/ajrfhp/anaconda3/envs/control/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/ajrfhp/Control/Control_FPL/data_processor.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_player_features["total_points"] = all_player_features["total_points"].clip(0, max_player_points)


(<torch.utils.data.dataloader.DataLoader at 0x7f97bb8e6280>,
 <torch.utils.data.dataloader.DataLoader at 0x7f96c1b8fd30>)

In [5]:
class LinearModelLightning(pl.LightningModule):
    def __init__(self, window_size=4, num_features=7, use_opponent_features=True, len_opponent_features=2):
        super().__init__()
        self.window_size = window_size
        self.dim = window_size * num_features
        self.use_opponent_features = use_opponent_features
        self.len_opponent_features = len_opponent_features
        self.fc1 = nn.Linear(self.dim, 1).double()
    
    def forward(self, x):
        x = x.reshape((x.shape[0], self.dim))
        return self.fc1(x).reshape((-1, ))
    
    def training_step(self, batch, batch_idx):
        x = batch[0]
        inputs = x[:,:,:self.window_size].reshape((x.shape[0], self.dim))
        outputs = x[:,0,self.window_size]
        predictions = self.fc1.forward(inputs)
        loss = nn.MSELoss()(predictions, outputs)
        self.log('train_loss', loss)
        return loss 
    
    def validation_step(self, batch, batch_idx):
        x = batch[0]
        inputs = x[:,:,:self.window_size].reshape((x.shape[0], self.dim))
        outputs = x[:,0,self.window_size]
        predictions = self.fc1.forward(inputs)
        loss = nn.MSELoss()(predictions, outputs)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [6]:
model = LinearModelLightning()
trainer = pl.Trainer(max_epochs=100)
trainer.fit(model, train_loader, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 29    
--------------------------------
29        Trainable params
0         Non-trainable params
29        Total params
0.000     Total estimated model params size (MB)


/home/ajrfhp/anaconda3/envs/control/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ajrfhp/anaconda3/envs/control/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([500])) that is different to the input size (torch.Size([500, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/ajrfhp/anaconda3/envs/control/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the 

Epoch 0:  79%|███████▉  | 45/57 [00:00<00:00, 128.21it/s, loss=1.16, v_num=12]

/home/ajrfhp/anaconda3/envs/control/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([344])) that is different to the input size (torch.Size([344, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1:  26%|██▋       | 15/57 [00:00<00:00, 121.36it/s, loss=1.1, v_num=12]

/home/ajrfhp/anaconda3/envs/control/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([87])) that is different to the input size (torch.Size([87, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 99: 100%|██████████| 57/57 [00:00<00:00, 109.69it/s, loss=0.977, v_num=12]
